In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
aulas_raw = pd.read_excel('/home/cayena/Downloads/rotas_geradas.xlsx')

In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

,professores_alocados,nome grupo
0,DANILO,ABU DHABI EAD - 4X
1,LUCIANO,ANDERSON VIP EAD - 1X
2,CENKAY,ANDRE VIP CONV EAD - 2X
3,LUCIANA,AUCKLAND P - 4X
4,CENKAY,BAKERSFIELD P - 4X
...,...,...
131,ALLAN TESTE,NEBRASKA P - 3X
132,DANILO,NORTHAMPTOM P - 3X
133,LEONEL,SAN JUAN EAD - 3X
134,ANDREY,SCOTT EAD - 3X
